In [ ]:
# Notebook 01 - INGESTÃO DE DADOS

# ETAPA 1: CONFIGURAÇÃO DO AMBIENTE

import pandas as pd
import numpy as np
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

pastas = ['../dados/bronze', '../dados/silver', '../dados/gold']
for pasta in pastas:
    os.makedirs(pasta, exist_ok=True)

print("Estrutura de pastas criada:")
print("   - dados/bronze  → Dados brutos")
print("   - dados/silver  → Dados transformados")
print("   - dados/gold    → Dados prontos para análise")



In [ ]:
# ETAPA 2: INGESTÃO DE DADOS (DATA INGESTION)

print("\n" + "="*60)
print("INICIANDO INGESTÃO DE DADOS")
print("="*60)

import os
import urllib.request

def obter_dataset():
    """
    Obtém o dataset usando múltiplos métodos:
    1. Verifica se já existe localmente
    2. Tenta download automático do GitHub
    3. Solicita caminho manual do arquivo
    """

    arquivos_possiveis = [
        'telco_churn.csv',
        'Telco-Customer-Churn.csv',
        'Telco_customer_churn.csv'
    ]

    for arquivo in arquivos_possiveis:
        if os.path.exists(arquivo):
            print(f"Arquivo encontrado: {arquivo}")
            return arquivo

    print("Arquivo não encontrado. Tentando download automático...\n")

    try:
        print("⬇Baixando dataset do GitHub...")
        url = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"
        nome_arquivo = 'telco_churn.csv'

        urllib.request.urlretrieve(url, nome_arquivo)

        if os.path.exists(nome_arquivo) and os.path.getsize(nome_arquivo) > 1000:
            print("Download concluído com sucesso!")
            return nome_arquivo
        else:
            raise Exception("Arquivo baixado está vazio ou corrompido")

    except Exception as e:
        print(f"Download automático falhou: {e}\n")

    print("Por favor, forneça o caminho completo do arquivo:")
    print("   Telco_customer_churn.csv")
    print("   (Digite o caminho completo do arquivo)\n")

    caminho_arquivo = input("Caminho do arquivo: ").strip()
    
    if os.path.exists(caminho_arquivo):
        print(f"\nArquivo encontrado: {caminho_arquivo}")
        return caminho_arquivo
    else:
        raise FileNotFoundError("Arquivo não encontrado no caminho fornecido!")

nome_arquivo = obter_dataset()

print(f"\nLendo arquivo: {nome_arquivo}")
df_bronze = pd.read_csv(nome_arquivo)

df_bronze['data_ingestao'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_bronze['fonte'] = 'Kaggle_TelcoChurn'

caminho_bronze = '../dados/bronze/telco_churn_raw.csv'
df_bronze.to_csv(caminho_bronze, index=False)

print(f"\nDados ingeridos com sucesso!")
print(f"   Total de registros: {len(df_bronze):,}")
print(f"   Arquivo salvo em: {caminho_bronze}")


In [ ]:
# ETAPA 3: EXPLORAÇÃO INICIAL DOS DADOS

print("\n" + "="*60)
print("ANÁLISE EXPLORATÓRIA INICIAL")
print("="*60)

print(f"\nDimensões do dataset: {df_bronze.shape}")
print(f"   - Linhas: {df_bronze.shape[0]:,}")
print(f"   - Colunas: {df_bronze.shape[1]}")

print("\nPrimeiras 5 linhas:")
print(df_bronze.head())

print("\nInformações dos tipos de dados:")
print(df_bronze.info())

print("\nEstatísticas descritivas:")
print(df_bronze.describe())

print("\nValores ausentes por coluna:")
missing = df_bronze.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("Nenhum valor ausente encontrado!")

print("\nDistribuição da variável alvo (Churn):")
print(df_bronze['Churn'].value_counts())
print(f"\n   Taxa de Churn: {(df_bronze['Churn']=='Yes').mean()*100:.2f}%")



In [ ]:

# ETAPA 4: VALIDAÇÕES DE QUALIDADE

print("\n" + "="*60)
print("VALIDAÇÕES DE QUALIDADE DOS DADOS")
print("="*60)

duplicatas = df_bronze.duplicated(subset='customerID').sum()
print(f"\nDuplicatas encontradas: {duplicatas}")

print("\nValores únicos em colunas categóricas:")
colunas_categoricas = ['gender', 'Partner', 'Dependents', 'PhoneService',
                       'InternetService', 'Contract', 'PaperlessBilling', 'Churn']
for col in colunas_categoricas:
    if col in df_bronze.columns:
        print(f"   - {col}: {df_bronze[col].unique()}")

print("\n📏 Range de valores numéricos:")
print(f"   - tenure: {df_bronze['tenure'].min()} a {df_bronze['tenure'].max()} meses")
print(f"   - MonthlyCharges: R$ {df_bronze['MonthlyCharges'].min():.2f} a R$ {df_bronze['MonthlyCharges'].max():.2f}")

print("\n" + "="*60)
print("INGESTÃO COMPLETA - Dados prontos para transformação!")
print("="*60)

relatorio = f"""
RELATÓRIO DE INGESTÃO DE DADOS
Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
==========================================
Fonte: {nome_arquivo}
Registros ingeridos: {len(df_bronze):,}
Colunas: {df_bronze.shape[1]}
Duplicatas: {duplicatas}
Taxa de Churn: {(df_bronze['Churn']=='Yes').mean()*100:.2f}%
==========================================
"""

with open('../dados/bronze/relatorio_ingestao.txt', 'w') as f:
    f.write(relatorio)

print("\nRelatório de ingestão salvo em: dados/bronze/relatorio_ingestao.txt")

In [ ]:
# ETAPA 5: SALVAR ARQUIVOS FINAIS

print("\n" + "="*60)
print("SALVANDO ARQUIVOS FINAIS")
print("="*60)

print("✅ Arquivos salvos com sucesso:")
print(f"Dados brutos: dados/bronze/telco_churn_raw.csv")
print(f"Relatório: dados/bronze/relatorio_ingestao.txt")

print("\nPróximos passos:")
print("   - Execute o notebook de transformação (02_transformacao.ipynb)")
print("   - Os dados estão prontos para análise!")